In [1]:
import pandas as pd
import re
import os

In [2]:
def load_data(input_file):
    print(f"\n=== Memproses file: {input_file} ===")
    try:
        # Membaca file CSV
        df = pd.read_csv(input_file)
        print(f"Data berhasil dimuat dengan ukuran {df.shape}")
        print(f"Jumlah baris sebelum dibersihkan: {len(df)}")
        return df
    except Exception as e:
        print(f"Terjadi kesalahan saat memuat file {input_file}: {e}")
        return None

In [3]:
def remove_duplicates(df):
    if df is None:
        return None
    
    # Membersihkan duplikat berdasarkan Link
    print(f"\n--- Menghapus data duplikat pada kolom 'Link' ---")
    duplicates = df[df.duplicated(subset=['Link'], keep=False)]
    print(f"Jumlah data duplikat: {len(duplicates)}")
    print(f"Jumlah baris sebelum menghapus duplikat: {len(df)}")
    
    # Hapus duplikat, simpan yang pertama
    df = df.drop_duplicates(subset=['Link'], keep='first')
    print(f"Jumlah baris setelah menghapus duplikat: {len(df)}")
    
    # Membersihkan duplikat berdasarkan Text
    print(f"\n--- Menghapus data duplikat pada kolom 'Text' ---")
    duplicates = df[df.duplicated(subset=['Text'], keep=False)]
    print(f"Jumlah data duplikat: {len(duplicates)}")
    print(f"Jumlah baris sebelum menghapus duplikat: {len(df)}")
    
    # Hapus duplikat, simpan yang pertama
    df = df.drop_duplicates(subset=['Text'], keep='first')
    print(f"Jumlah baris setelah menghapus duplikat: {len(df)}")
    
    return df

In [4]:
def clean_data(df):
    if df is None:
        return None
    
    print(f"\n--- Membersihkan karakter spesifik pada kolom 'Text' ---")
    print("\nHead data sebelum membersihkan teks:")
    print(df.head())
    
    # Fungsi untuk membersihkan teks
    def clean_text(text):
        if isinstance(text, str):
            # Hapus karakter spesifik: @, #, $, &, *, ~
            text = re.sub(r'[@#$&*~]', '', text)
            
            # Hapus kurung kurawal {} dan kurung siku []
            text = re.sub(r'[\{\}\[\]]', '', text)
            
            # Hapus +, =, <, >
            text = re.sub(r'[\+=<>]', '', text)
            
            # Hapus emoji (rentang karakter Unicode untuk emoji)
            emoji_pattern = re.compile("["
                                      u"\U0001F600-\U0001F64F"  # emoticons
                                      u"\U0001F300-\U0001F5FF"  # simbol & piktograf
                                      u"\U0001F680-\U0001F6FF"  # simbol transportasi & peta
                                      u"\U0001F700-\U0001F77F"  # alethiometer
                                      u"\U0001F780-\U0001F7FF"  # simbol geometris
                                      u"\U0001F800-\U0001F8FF"  # simbol tambahan
                                      u"\U0001F900-\U0001F9FF"  # simbol tambahan 2
                                      u"\U0001FA00-\U0001FA6F"  # simbol tambahan 3
                                      u"\U0001FA70-\U0001FAFF"  # simbol tambahan 4
                                      u"\U00002702-\U000027B0"  # dingbats
                                      u"\U000024C2-\U0001F251" 
                                      "]+", flags=re.UNICODE)
            text = emoji_pattern.sub(r'', text)
            
            # Hapus semua karakter non-alfabet (kecuali spasi)
            text = re.sub(r'[^a-zA-Z\s]', '', text)
            
            # Hapus spasi berlebih
            text = re.sub(r'\s+', ' ', text).strip()
            
            return text
        return text
    
    # Terapkan fungsi pembersihan
    df['Text'] = df['Text'].apply(clean_text)
    
    print("\nHead data setelah membersihkan teks:")
    print(df.head())
    
    return df

In [5]:
def export_data(df, output_file):
    if df is None:
        return False
    
    try:
        # Buat direktori output jika belum ada
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Simpan ke CSV
        df.to_csv(output_file, index=False)
        print(f"\nData yang sudah dibersihkan berhasil disimpan ke '{output_file}'")
        print(f"Ukuran akhir data: {df.shape}")
        
        return True
    except Exception as e:
        print(f"Terjadi kesalahan saat menyimpan file {output_file}: {e}")
        return False

In [6]:
def process_single_file(input_file, output_file):
    # Load data
    df = load_data(input_file)
    
    # Remove duplicates
    df = remove_duplicates(df)
    
    # Clean data
    df = clean_data(df)
    
    # Export data
    result = export_data(df, output_file)
    
    return result

In [7]:
# Array file yang akan diproses
files = [
    {'input': 'scrap/jatim_prov.csv', 'output': 'cleaned/jatim_prov_clean.csv'},
    {'input': 'scrap/kalimantan_utara.csv', 'output': 'cleaned/kalimantan_utara_clean.csv'},
    {'input': 'scrap/kalbar.csv', 'output': 'cleaned/kalbar_clean.csv'},
    {'input': 'scrap/salatiga_kota.csv', 'output': 'cleaned/salatiga_kota_clean.csv'},
    {'input': 'scrap/blitar_kota.csv', 'output': 'cleaned/blitar_kota_clean.csv'},
    {'input': 'scrap/depok_kota.csv', 'output': 'cleaned/depok_kota_clean.csv'},
    {'input': 'scrap/malang_kota.csv', 'output': 'cleaned/malang_kota_clean.csv'},
    {'input': 'scrap/semarang_kota.csv', 'output': 'cleaned/semarang_kota_clean.csv'},
    {'input': 'scrap/madiun_kota.csv', 'output': 'cleaned/madiun_kota_clean.csv'},
    {'input': 'scrap/baubau_kota.csv', 'output': 'cleaned/baubau_kota_clean.csv'},
    {'input': 'scrap/sleman_kab.csv', 'output': 'cleaned/sleman_kab_clean.csv'},
    {'input': 'scrap/mahakamulu_kab.csv', 'output': 'cleaned/mahakamulu_kab_clean.csv'},
    {'input': 'scrap/bengkalis_kab.csv', 'output': 'cleaned/bengkalis_kab_clean.csv'},
    {'input': 'scrap/ngada_kab.csv', 'output': 'cleaned/ngada_kab_clean.csv'},
    {'input': 'scrap/rejanglebong_kab.csv', 'output': 'cleaned/rejanglebong_kab_clean.csv'},
    {'input': 'scrap/ngawi_kab.csv', 'output': 'cleaned/ngawi_kab_clean.csv'},
    {'input': 'scrap/deliserdang_kab.csv', 'output': 'cleaned/deliserdang_kab_clean.csv'},
    {'input': 'scrap/lampungtengah_kab.csv', 'output': 'cleaned/lampungtengah_kab_clean.csv'},
    {'input': 'scrap/puncak_jaya_kab.csv', 'output': 'cleaned/puncak_jaya_kab_clean.csv'},
    {'input': 'scrap/bombana_kab.csv', 'output': 'cleaned/bombana_kab_clean.csv'},
    # Tambahkan file lain jika diperlukan
]

In [8]:
def process_multiple_files(files):
    total_files = len(files)
    successful = 0
    
    for i, file_info in enumerate(files):
        input_file = file_info['input']
        output_file = file_info['output']
        print(f"\nMemproses file {i+1}/{total_files}: {input_file}")
        
        if process_single_file(input_file, output_file):
            successful += 1
    
    print(f"\n=== Ringkasan Proses ===")
    print(f"Total file diproses: {total_files}")
    print(f"Berhasil: {successful}")
    print(f"Gagal: {total_files - successful}")

In [9]:
process_multiple_files(files)


Memproses file 1/20: scrap/jatim_prov.csv

=== Memproses file: scrap/jatim_prov.csv ===
Data berhasil dimuat dengan ukuran (5785, 2)
Jumlah baris sebelum dibersihkan: 5785

--- Menghapus data duplikat pada kolom 'Link' ---
Jumlah data duplikat: 34
Jumlah baris sebelum menghapus duplikat: 5785
Jumlah baris setelah menghapus duplikat: 5768

--- Menghapus data duplikat pada kolom 'Text' ---
Jumlah data duplikat: 566
Jumlah baris sebelum menghapus duplikat: 5768
Jumlah baris setelah menghapus duplikat: 5220

--- Membersihkan karakter spesifik pada kolom 'Text' ---

Head data sebelum membersihkan teks:
                                                Link  \
0  https://jatimprov.go.id/berita/indeks-spbe-cap...   
1  https://jatimprov.go.id/berita/gubernur-khofif...   
2  https://jatimprov.go.id/berita/kendalikan-infl...   
3  https://jatimprov.go.id/berita/sepanjang-2021-...   
4  https://jatimprov.go.id/berita/gubernur-khofif...   

                                                Text  
0 